<h1>SASCalc</h1>

<b>Purpose:</b>
This notebook replicates the calculations performed by the NCNR Igor macro SASCalc. The user inputs parameters of the small angle neutron scattering instrument, and the notebook calculates the scattering as a function of q vector in 2-D and 1-D. Unlike the Igor macro version, the notebook always calculates the 2-D version, and derives the 1-D data from an averaging method. The Igor version defaulted to a straight 1-D calculation, though the resolution was always done in 2-D.

<b>Input Classes:</b><ul>
<li>Source: This is a neutron flux value at the end of the non-programmable neutron guides.</li>
<li>Wavelength: This class describes the wavelength properties of the beam after a velocity selector or monochromator. Included in the description are the wavelength, the spread in wavelength, and the type of device. For a velocity selector, a velocity is calculated and stored in the class object.</li>
<li>Attenuator: This class describes the effects of an attenuator on the beam intensity. It does not affect beam divergence, wavelength, or wavelength spread.</li>
<li>Source Aperture: This class describes the source aperture. Included in the description are its coordinates, its shape (circular or rectangular), and dimensions (radius or width-height). A flux after the aperture is a calculated number.</li>
<li>Guide: This class describes the configuration of programmable guides. Included in the description are the number of guides.</li>
<li>Sample Aperture: This class describes the source aperture. Included in the description are its coordinates, its shape (circular or rectangular), and dimensions (radius or width-height). A flux after the aperture is a calculated number.</li>
<li>Sample: This class describes the sample, which is represented in fourier space by a SASview scattering model. The class includes the sample coordinates, thickness, cell details, and parameters required to calculate the scattering model.
<li>Detector: This class describes the 2-dimensional neutron detector. Included in the description are its coordinates, its dimensions (number of pixels, pixel size), and performance (dead time, pixel blur). The counts on the detector are calculated in 2D and 1D.</li>

In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import sys
sys.path.insert(0,'./')
from components import *
from instrument import *


<h2>Load instrument components</h2>
<b>Purpose:</b> Load all of the component classes with initial parameters<br><br>

In [22]:
src = Source(loc=-800, flux=100000)
velsel = VelocitySelector(loc=-600, wavelen=5, spread=0.14)
atten = Attenuator(loc=-510, number=0, factor=0)
srcap = Aperture(loc=-500, shape='circle', dims=[2.54, 2.54])
samap = Aperture(loc=0, shape='circle', dims=[0.635, 0.635])
guides = Guide(loc=-500, number=0, dims=[5,5], parms=[2,2])
sam = Sample(loc=5, label='Cylinders in d2O', dims=[2.54, 2.54, 0.254], model='cylinder')
bs = BeamStop(loc=495, coords=[0.2, 5.3], bsnum=2, bsdims=[5.08, 5.08], A=20.1, B=2.0477)
det = Detector(loc=500, dpix=[0.508, 0.508], npix=[128,128], beam_cntr=[64.1, 65.2])

In [25]:
sans = Instrument(src, velsel, atten, guides, srcap, samap, sam, bs, det)

In [ ]:
## Use SASModels to calculate a smeared function on the detector
## 1) fill in a dictionary of parameters for the model
## 2) Create a sasmodels:Data2D object either with empty_data2D or with data2D class directly
## 3) Create a kernel object based on the model name
## 4) Create an instance of the sasmodels:DirectModel class based on the data2D and kernel objects
## 5) Caculate the 2D array of intensity through the DirectModel instance

pars = {'radius': 200, 'radius_pd': 0.1, 'scale': 2}
res = 0.05
Qx, Qy = np.meshgrid(det.ax_qx, det.ax_qy)
Qx, Qy = Qx.flatten(), Qy.flatten()
Q = np.sqrt(Qx**2 + Qy**2)
dqx = res * Q
dqy = res * Q
Iq = 100 * np.ones_like(Qx)
dIq = np.sqrt(Iq)
data = Data2D(x=Qx, y=Qy, z=Iq, dx=dqx, dy=dqy, dz=dIq)
kernel = load_model(sam.model)
f = DirectModel(data, kernel)
Iq = np.reshape(f(**pars), (128, 128))

In [ ]:
plt.imshow(np.log10(sans10m.Iq))
plt.title('Detector Image')
plt.colorbar()
plt.ylabel('Y Pixels')
plt.xlabel('X Pixels')
